In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Define the transforms for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])


In [3]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)


100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [02:32<00:00, 1118380.49it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:

# Define DataLoader to handle batches and shuffling
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)


In [16]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)  # 10 classes in CIFAR-10

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [17]:
# Initialize the model, loss function, and optimizer
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [18]:

# Training the model
num_epochs = 10
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/782], Loss: 1.6766
Epoch [1/10], Step [200/782], Loss: 1.4352
Epoch [1/10], Step [300/782], Loss: 1.4589
Epoch [1/10], Step [400/782], Loss: 1.4400
Epoch [1/10], Step [500/782], Loss: 1.4611
Epoch [1/10], Step [600/782], Loss: 1.0824
Epoch [1/10], Step [700/782], Loss: 1.4276
Epoch [2/10], Step [100/782], Loss: 1.3629
Epoch [2/10], Step [200/782], Loss: 1.0214
Epoch [2/10], Step [300/782], Loss: 1.1302
Epoch [2/10], Step [400/782], Loss: 1.1751
Epoch [2/10], Step [500/782], Loss: 0.9303
Epoch [2/10], Step [600/782], Loss: 1.0169
Epoch [2/10], Step [700/782], Loss: 1.0261
Epoch [3/10], Step [100/782], Loss: 1.0532
Epoch [3/10], Step [200/782], Loss: 0.8294
Epoch [3/10], Step [300/782], Loss: 0.8660
Epoch [3/10], Step [400/782], Loss: 0.7107
Epoch [3/10], Step [500/782], Loss: 0.9715
Epoch [3/10], Step [600/782], Loss: 0.6831
Epoch [3/10], Step [700/782], Loss: 0.9279
Epoch [4/10], Step [100/782], Loss: 1.0556
Epoch [4/10], Step [200/782], Loss: 0.8353
Epoch [4/10

In [19]:
# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {(100 * correct / total):.2f}%')

Accuracy of the model on the test images: 69.62%
